In [23]:
import re
import sys
import pprint
import json
from IPython.display import display
from colorama import Fore, Back, Style
import subprocess

In [53]:
subprocess.run(["python3", "lexical-analyzer.py"])
with open('./results/lab1.json') as lab1_file:
    data = json.load(lab1_file)
    
tokens_chain = data['chain']
service_words = data['tables']['service_words']
operations = data['tables']['operations']
separators = data['tables']['separators']
constants = data['tables']['constants']
identifiers = data['tables']['identifiers']

nxtsymb = []
cursymb = []
token_index = -1
result = wrapper()
if result:
    error(result)

['W', 0, 'program', 1]
program
['I', 0, 'testovaya', 1]
['R', 4, ';', 1]
['W', 1, 'var', 3]
['I', 1, 'b', 3]
var
['R', 1, ',', 3]
['I', 2, 'c', 3]
['R', 1, ',', 3]
['I', 3, 'd', 3]
['R', 3, ':', 3]
['W', 3, 'integer', 3]
var_type
['R', 4, ';', 3]
['I', 4, 'a', 4]
var
['R', 1, ',', 4]
['I', 5, 'e', 4]
['R', 1, ',', 4]
['I', 6, 'f', 4]
['R', 1, ',', 4]
['I', 7, 'g', 4]
['R', 3, ':', 4]
['W', 4, 'real', 4]
var_type
['R', 4, ';', 4]
['I', 8, 's1', 5]
var
['R', 1, ',', 5]
['I', 9, 's2', 5]
['R', 1, ',', 5]
['I', 10, 's3', 5]
['R', 1, ',', 5]
['I', 11, 's4', 5]
['R', 3, ':', 5]
['W', 5, 'string', 5]
var_type
['R', 4, ';', 5]
['I', 12, 'super_array', 6]
var
['R', 3, ':', 6]
['W', 7, 'array', 6]
var_type
['R', 7, '[', 6]
['C', 0, '1', 6]
interval
['R', 2, '..', 6]
['C', 1, '3', 6]
['R', 1, ',', 6]
['C', 2, '10', 6]
interval
['R', 2, '..', 6]
['C', 3, '20', 6]
['R', 8, ']', 6]
['W', 8, 'of', 6]
['W', 3, 'integer', 6]
['R', 4, ';', 6]
['I', 13, 'str', 7]
var
['R', 3, ':', 7]
['W', 5, 'string', 7

In [2]:
def scan():
    global tokens_chain, token_index, nxtsymb, cursymb
    token_index += 1
    if token_index < len(tokens_chain):
        cursymb = nxtsymb
        nxtsymb = tokens_chain[token_index]
        print(nxtsymb)

In [3]:
def antiscan():
    global tokens_chain, token_index, nxtsymb
    token_index -= 1
    if token_index >= 0:
        nxtsymb = tokens_chain[token_index]
        print(nxtsymb)

In [33]:
def error(text):
    global cursymb
    print(Fore.RED + 'Fatal:', Fore.BLACK + text, Style.RESET_ALL + Fore.WHITE + 'at line', Style.RESET_ALL + str(nxtsymb[3]))

In [50]:
def check(response):
    if response:
        error(response)
        return False
    else:
        return True

In [52]:
def wrapper():
    global nxtsymb, cursymb
    scan()
    if nxtsymb[2] == 'program':
        if not check(program()):
            return 'invalid program declaration'
        
        scan()
    if nxtsymb[2] == 'var':
        scan()
        if not check(var()):
            return 'invalid var declaration'
        scan()
        if nxtsymb[2] != ';':
            return '";" expected'
        scan()
        while nxtsymb[0] == 'I':
            if not check(var()):
                return 'invalid var declaration'
            scan()
            if nxtsymb[2] != ';':
                return '";" expected'
            scan()
    
#     scan()
    while nxtsymb[2] in ['function', 'procedure']:
        if not check(function_or_procedure()):
            return 'invalid function or procedure declaration'
    
#     scan()
    if nxtsymb[2] != 'begin':
        return '"begin" expected'
    scan()
    while len(operator()) == 0:
        scan()
        if cursymb[2] == ':':
            scan()
        if nxtsymb[2] != ';':
            return '";" expected instead "' + str(nxtsymb[2]) + '"'
            
#     print('END', nxtsymb[2])
    if nxtsymb[2] != 'end.':
        if cursymb[2] == ';' and nxtsymb[2] == 'else':
            return 'extra ";" before else'
        return 'unexpected "'+nxtsymb[2]+'"'
    

In [7]:
def program():
    global nxtsymb
    print(Fore.GREEN + 'program' + Style.RESET_ALL)
    
    scan()
    if nxtsymb[0] != 'I':
        return 'identifier expected'
    scan()
    if nxtsymb[2] != ';':
        return '";" expected'
    return ''

In [8]:
def var():
    global nxtsymb
    print(Fore.GREEN + 'var' + Style.RESET_ALL)
    
    if nxtsymb[0] != 'I':
        return 'identifier expexted'
    scan()
    while nxtsymb[2] == ',':
        scan()
        if nxtsymb[0] != 'I':
            return 'identifier expected'
        scan()
    if nxtsymb[2] != ':':
        return '":" expected'
    scan()
    if not check(var_type()):
        return 'invalid variable type declaration'
    return ''

In [9]:
def var_type():
    global nxtsymb
    print(Fore.GREEN + 'var_type' + Style.RESET_ALL)
    
    if nxtsymb[2] in ['integer', 'real', 'string']:
        return ''
    elif nxtsymb[2] == 'array':
        scan()
        if nxtsymb[2] != '[':
            return '"[" expected'
        scan()
        if not check(interval()):
            return 'invalid interval syntax'
        scan()
        while nxtsymb[2] == ',':
            scan()
            if not check(interval()):
                return 'invalid interval syntax'
            scan()
        if nxtsymb[2] != ']':
            return '"]" expected'
        scan()
        if nxtsymb[2] != 'of':
            return '"of" expected'
        scan()
        if nxtsymb[2] not in ['integer', 'real', 'string']:
            return 'variable type expected'
        return ''
    else:
        return 'array type expected'
        
            

In [10]:
def interval():
    global nxtsymb
    print(Fore.GREEN + 'interval' + Style.RESET_ALL)
    
    if nxtsymb[0] != 'C':
        return 'constant expected'
    scan()
    if nxtsymb[2] != '..':
        return '".." expected'
    scan()
    if nxtsymb[0] != 'C':
        return 'constant expected'
    return ''

In [11]:
def function_or_procedure():
    global nxtsymb
    print(Fore.GREEN + 'function_or_procedure' + Style.RESET_ALL)
    
    if nxtsymb[2] == 'procedure':
        scan()
        if not check(procedure()):
            return 'invalid procedure declaration'
        return ''
    if nxtsymb[2] == 'function':
        scan()
        if not check(function()):
            return 'invalid function declaration'
        return ''
    return ''        

In [12]:
def procedure():
    global nxtsymb
    print(Fore.GREEN + 'procedure' + Style.RESET_ALL)
    
    if nxtsymb[0] != 'I':
        return 'identifier expended'
    scan()
    if nxtsymb[2] == '(':
        scan()
        if not check(var()):
            return 'invalid var declaration'
        scan()
        while nxtsymb[2] == ';':
            scan()
            if not check(var()):
                return 'invalid var declaration'
            scan()
        if nxtsymb[2] != ')':
            return '")" expected'
        scan()
    if nxtsymb[2] != ';':
        return '";" expected'
    
    scan()
    if nxtsymb[2] == 'var':
        scan()
        if not check(var()):
            return 'invalid var declaration'
        scan()
        if nxtsymb[2] != ';':
            return '";" expected'
        scan()
        while nxtsymb[0] == 'I':
            if not check(var()):
                return 'invalid var declaration'
            scan()
            if nxtsymb[2] != ';':
                return '";" expected'
            scan()
    
    while nxtsymb[2] in ['function', 'procedure']:
        if not check(function_or_procedure()):
            return 'invalid function or procedure declaration'
    
#     scan()
    if nxtsymb[2] != 'begin':
        return '"begin" expected'
    scan()
    while len(operator()) == 0:
        scan()
        if cursymb[2] != ':':
            if nxtsymb[2] != ';':
                return '";" expected'
            scan()
    if nxtsymb[2] != 'end':
        if cursymb[2] == ';' and nxtsymb[2] == 'else':
            return 'extra ";" before else'
        return 'unexpected "'+nxtsymb[2]+'"'
    scan()
    if nxtsymb[2] != ';':
        return '";" expected'
    scan()
    return ''

In [13]:
def function():
    global nxtsymb
    print(Fore.GREEN + 'function' + Style.RESET_ALL)
    
    if nxtsymb[0] != 'I':
        return 'identifier expended'
    scan()
    if nxtsymb[2] == '(':
        scan()
        if not check(var()):
            return 'invalid var declaration'
        scan()
        while nxtsymb[2] == ';':
            scan()
            if not check(var()):
                return 'invalid var declaration'
            scan()
        if nxtsymb[2] != ')':
            return '")" expected'
        scan()
    if nxtsymb[2] != ':':
        return 'expected ":"'
    scan()
    if nxtsymb[2] not in ['integer', 'real', 'string']:
        return 'type of function expected'
    scan()
    if nxtsymb[2] != ';':
        return '";" expected'
    
    scan()
    if nxtsymb[2] == 'var':
        scan()
        if not check(var()):
            return 'invalid var declaration'
        scan()
        if nxtsymb[2] != ';':
            return '";" expected'
        scan()
        while nxtsymb[0] == 'I':
            if not check(var()):
                return 'invalid var declaration'
            scan()
            if nxtsymb[2] != ';':
                return '";" expected'
            scan()
    
    while nxtsymb[2] in ['function', 'procedure']:
        if not check(function_or_procedure()):
            return 'invalid function or procedure declaration'
    
#     scan()
    if nxtsymb[2] != 'begin':
        return '"begin" expected'
    
    scan()
    while len(operator()) == 0:
        scan()
        if cursymb[2] != ':':
            if nxtsymb[2] != ';':
                return '";" expected'
            scan()
    
    if nxtsymb[2] != 'return':
        return 'function have to return some value'
    scan()
    if not check(expression()):
        return 'return\'s exression expected'
    scan()
    if nxtsymb[2] != ';':
        return '";" expected'
    
    scan()
    while len(operator_and_return()) == 0:
        scan()
        if cursymb[2] != ':':
            if nxtsymb[2] != ';':
                return '";" expected'
            scan()
            
    if nxtsymb[2] != 'end':
        if cursymb[2] == ';' and nxtsymb[2] == 'else':
            return 'extra ";" before else'
        return 'unexpected "'+nxtsymb[2]+'"'
    scan()
    if nxtsymb[2] != ';':
        return '";" expected'
    scan()
    return ''

In [39]:
def operator():
    global nxtsymb
    print(Fore.GREEN + 'operator' + Style.RESET_ALL)

    if nxtsymb[2] == ';':
        pass
    elif nxtsymb[0] == 'I':
        scan()
        if nxtsymb[2] == ':':
            return ''
        if nxtsymb[2] == '[':
            scan()
            if not check(expression()):
                return 'invalid array arguments'
            scan()
            while nxtsymb[2] == ',':
                scan()
                if not check(expression()):
                    return 'invalid array arguments'
                scan()
            if nxtsymb[2] != ']':
                return '\']\' expected'
            scan()
        if nxtsymb[2] == ':=':
            if not check(assignment()):
                return 'invalid assignment declaration'
            return ''
        if nxtsymb[2] == '(':
            scan()
            if not check(expression()):
                return 'invalid argument'
            scan()
            while nxtsymb[2] == ',':
                scan()
                if not check(expression()):
                    return 'invalid argument'
                scan()
            if nxtsymb[2] != ')':
                return 'expected \')\''
            return ''
        return 'unexpected "'+nxtsymb[2]+'"'
    elif nxtsymb[2] == 'if':
        scan()
        if not check(condition()):
            return 'condition expected'
        scan()
        if nxtsymb[2] != 'then':
            return '"then" expected'
        scan()
        if not check(operator()):
            return 'operator expected'
        scan()
        if nxtsymb[2] == 'else':
            scan()
            if not check(operator()):
                return 'operator expected'
        else:
            scan()
            if nxtsymb[2] == 'else':
                return 'extra ";" before "else"'
    elif nxtsymb[2] == 'goto':
        scan()
        if nxtsymb[0] != 'I':
            return 'identifier expected'
    elif nxtsymb[2] == 'while':
        scan()
        if not check(condition()):
            return 'condition expected'
        scan()
        if nxtsymb[2] != 'do':
            return '"do" expected'
        scan()
        if not check(operator()):
            return 'operator expected'
    else:
        return 'operator expexted'
    return ''

In [40]:
def operator_and_return():
    global nxtsymb
    print(Fore.GREEN + 'operator' + Style.RESET_ALL)

    if nxtsymb[2] == ';':
        pass
    elif nxtsymb[0] == 'I':
        scan()
        if nxtsymb[2] == ':':
            return ''
        if nxtsymb[2] == '[':
            scan()
            if not check(expression()):
                return 'invalid array arguments'
            scan()
            while nxtsymb[2] == ',':
                scan()
                if not check(expression()):
                    return 'invalid array arguments'
                scan()
            if nxtsymb[2] != ']':
                return '"]" expected'
            scan()
        if nxtsymb[2] == ':=':
            if not check(assignment()):
                return 'invalid assignment declaration'
            return ''
        if nxtsymb[2] == '(':
            scan()
            if not check(expression()):
                return 'invalid argument'
            scan()
            while nxtsymb[2] == ',':
                scan()
                if not check(expression()):
                    return 'invalid argument'
                scan()
            if nxtsymb[2] != ')':
                return 'expected ")"'
            return ''
        return 'unexpected "'+nxtsymb[2]+'"'
    elif nxtsymb[2] == 'if':
        scan()
        if not check(condition()):
            return 'condition expected'
        scan()
        if nxtsymb[2] != 'then':
            return '"then" expected'
        scan()
        if not check(operator()):
            return 'operator expected'
        scan()
        if nxtsymb[2] == 'else':
            scan()
            if not check(operator()):
                return 'operator expected'
        else:
            antiscan()
    elif nxtsymb[2] == 'goto':
        scan()
        if nxtsymb[0] != 'I':
            return 'identifier expected'
    elif nxtsymb[2] == 'while':
        scan()
        if not check(condition()):
            return 'condition expected'
        scan()
        if nxtsymb[2] != 'do':
            return '"do" expected'
        scan()
        if not check(operator()):
            return 'operator expected'
    elif nxtsymb[2] == 'return':
        scan()
        if not check(expression()):
            return 'return\'s exression expected'
    else:
        return 'operator expected'
    return ''

In [16]:
def condition():
    global nxtsymb
    print(Fore.GREEN + 'condition' + Style.RESET_ALL)
    
    if not check(expression()):
        return 'expression expected'
    scan()
    if nxtsymb[2] not in ['=', '>', '<', '>=', '<=', '<>']:
        return 'comparasion operation'
    scan()
    if not check(expression()):
        return 'expression expected'
    return ''

In [17]:
def assignment():
    global nxtsymb
    print(Fore.GREEN + 'assignment' + Style.RESET_ALL)
    
    if nxtsymb[2] != ':=':
        return '":=" expected'
    scan()
    if not check(expression()):
        return 'invalid expression'
    print(Fore.GREEN + 'expression checked' + Style.RESET_ALL)
    return ''

In [18]:
def expression():
    print(Fore.GREEN + 'expression' + Style.RESET_ALL)
    global nxtsymb
    
#     print('Terms:')
    if not check(term()):
        return 'term expected'
#     print(Fore.BLUE + 'Term:' + Style.RESET_ALL, nxtsymb[2])
    scan()
    while nxtsymb[2] in ['+', '-']:
        scan()
        if not check(term()):
            return 'term expected'
#         print(Fore.BLUE + 'Term:' + Style.RESET_ALL, nxtsymb[2])
        scan()
    else:
        antiscan()
        pass
        
    return ''

In [19]:
def term():
    global nxtsymb
    print(Fore.GREEN + 'term' + Style.RESET_ALL)
    
    if not check(factor()):
        return 'factor expected'
#     print(Fore.BLUE + 'Factor:' + Style.RESET_ALL, nxtsymb[2])
    scan()
    while nxtsymb[2] in ['*', '/', '^']:
        scan()
        if not check(factor()):
            return 'factor expected'
#         print(Fore.BLUE + 'Factor:' + Style.RESET_ALL, nxtsymb[2])
        scan()
    else:
        antiscan()
        pass
#         print('End of factors')
        
    return ''

In [20]:
def factor():
    global nxtsymb
    print(Fore.GREEN + 'factor' + Style.RESET_ALL)
    
    if nxtsymb[2] == '(':
        scan()
        if not check(expression()):
            return 'expression expected'
        scan()
        if nxtsymb[2] != ')':
            return '")" expected'
    else:
        if not check(argument()):
            return 'constant or identifier expected'
#         print('heh')
    return ''

In [21]:
def argument():
    global nxtsymb
    print(Fore.GREEN + 'argument' + Style.RESET_ALL)

    if nxtsymb[0] not in ['I', 'C']:
        return 'constant or identifier expected'
    scan()
    if nxtsymb[2] in ['[', '(']:
        scan()
        if not(check(expression())):
            return 'invalid expression'
        scan()
        while nxtsymb[2] == ',':
            scan()
            if not(check(expression())):
                return 'invalid expression'
            scan()
        if nxtsymb[2] not in [']', ')']:
            return 'expected \']\' or \')\''
    else:
        antiscan()
    print(Fore.BLUE + 'Argument:' + Style.RESET_ALL, nxtsymb[2])
    return ''